In [9]:
# Cell 1 – imports and config

import gc
import numpy as np
import pandas as pd
import cudf

import os
os.environ['HM_TARGET_WEEK_END'] = '2020-08-19'
TARGET_WEEK_END = os.environ.get("HM_TARGET_WEEK_END")
printdate = TARGET_WEEK_END
assert TARGET_WEEK_END is not None, "Set HM_TARGET_WEEK_END=YYYY-MM-DD"
TARGET_WEEK_END = pd.to_datetime(TARGET_WEEK_END)

N_SIM_ITEMS = 1000    # number of similar items per item
N_PER_USER = 170    # max itemCF candidates per user
MIN_COOC = 3     # min co-occurrence count to keep a pair

In [10]:
# Cell 2 – load transactions

tx = cudf.read_csv(
    '../data/input_data/transactions_train.csv',
    usecols=['t_dat', 'customer_id', 'article_id'],
    dtype={'t_dat': 'string', 'customer_id': 'string', 'article_id': 'int32'},
)

# Same int customer_id mapping as other pipelines
tx['customer_id'] = tx['customer_id'].str[-16:].str.hex_to_int().astype('int64')
tx['t_dat'] = cudf.to_datetime(tx['t_dat'])

last_ts = TARGET_WEEK_END

cut_ts = last_ts - pd.Timedelta(days=7)
history_start = cut_ts - pd.Timedelta(days=45) 

tx = tx[(tx['t_dat'] > history_start) & (tx['t_dat'] <= cut_ts)]

tx.head()

,t_dat,customer_id,article_id
28287015,2020-06-29,1249760199313500820,806964002
28287016,2020-06-29,1249760199313500820,806964002
28287017,2020-06-29,1249760199313500820,812207002
28287018,2020-06-29,8282976502227659861,733749001
28287019,2020-06-29,8282976502227659861,875329001


In [11]:
# Cell 3 – unique user–item interactions

ui = tx[['customer_id', 'article_id']].drop_duplicates()
ui.head()

,customer_id,article_id
28287015,1249760199313500820,806964002
28287017,1249760199313500820,812207002
28287018,8282976502227659861,733749001
28287019,8282976502227659861,875329001
28287020,-6382781975245111187,841434007


In [12]:
# Cell 4 – item-item co-occurrence counts

# Self-join on customer_id to get (item_i, item_j) pairs for the same user
pairs = ui.merge(
    ui,
    on='customer_id',
    how='inner',
    suffixes=('_i', '_j')
)

# Drop self pairs
pairs = pairs[pairs['article_id_i'] != pairs['article_id_j']]

# Enforce ordering to avoid (i,j) and (j,i) duplicates
mask = pairs['article_id_i'] < pairs['article_id_j']
pairs = pairs[mask]

# Count co-occurrences
cooc = (
    pairs
    .groupby(['article_id_i', 'article_id_j'])
    .size()
    .reset_index(name='coocount')
)

# Filter weak pairs
cooc = cooc[cooc['coocount'] >= MIN_COOC]

cooc.head()

,article_id_i,article_id_j,coocount
45,372860069,765773007,6
57,850732003,859942001,3
60,556539001,816166005,5
111,448509014,878483002,3
118,834906012,873072002,3


In [13]:
# Cell 5 – similarity and top-N similar per item

# Item frequencies (number of distinct users)
item_freq = ui.groupby('article_id').size().reset_index(name='freq')
item_freq = item_freq.rename(columns={'article_id': 'article_id_i'})

cooc = cooc.merge(item_freq, on='article_id_i', how='left')
item_freq_j = item_freq.rename(columns={
    'article_id_i': 'article_id_j',
    'freq': 'freq_j'
})
cooc = cooc.merge(item_freq_j, on='article_id_j', how='left')

# Simple cosine-like similarity
cooc['sim'] = cooc['coocount'] / (
    (cooc['freq'] * cooc['freq_j']) ** 0.5
)

# For each article_i, keep top N_SIM_ITEMS similar items
cooc = cooc.sort_values(['article_id_i', 'sim'], ascending=[True, False])
cooc['rank'] = cooc.groupby('article_id_i')['sim'].rank(method='first', ascending=False)
cooc = cooc[cooc['rank'] <= N_SIM_ITEMS]

# (article_i -> similar article_j, with similarity score)
item_sim = cooc[['article_id_i', 'article_id_j', 'sim']].rename(
    columns={
        'article_id_i': 'article_id',
        'article_id_j': 'sim_article_id',
        'sim': 'sim_score',
    }
)

item_sim.head()

,article_id,sim_article_id,sim_score
278001,108775044,767834002,0.054841
262707,108775044,887659010,0.046595
44102,108775044,841383003,0.016265
288423,108775044,749699002,0.015583
227064,108775044,736870001,0.014308


In [14]:
# Cell 6 – recent items per user

K_RECENT = 30  # how many recent items per user to base neighbors on

tx_sorted = tx.sort_values(['customer_id', 't_dat'], ascending=[True, False])
tx_sorted['rnk'] = tx_sorted.groupby('customer_id').cumcount() + 1
recent = tx_sorted[tx_sorted['rnk'] <= K_RECENT][['customer_id', 'article_id']].drop_duplicates()

recent.head()

,customer_id,article_id
30235723,-9223343869995384291,783346032
30235724,-9223343869995384291,849886012
30235726,-9223343869995384291,865929003
30235727,-9223343869995384291,849886010
28993786,-9223343869995384291,927529004


In [15]:
# Cell 7 – itemCF candidates per user (BATCHED to avoid OOM). Use N_SIM_ITEMS=350 & N_PER_USER=350 above.
BATCH_SIZE = 20000  # lower (e.g. 8000) if still OOM

recent_users = recent['customer_id'].unique()
out_parts = []

for start in range(0, len(recent_users), BATCH_SIZE):
    u_slice = recent_users[start:start + BATCH_SIZE]
    rb = recent.merge(cudf.DataFrame({'customer_id': u_slice}), on='customer_id', how='inner')

    cand_part = rb.merge(item_sim, on='article_id', how='left')
    cand_part = cand_part.dropna(subset=['sim_article_id'])

    cand_part = (
        cand_part
        .groupby(['customer_id', 'sim_article_id'])['sim_score']
        .sum()
        .reset_index()
        .rename(columns={'sim_article_id': 'article_id', 'sim_score': 'value'})
    )

    cand_part = cand_part.sort_values(['customer_id', 'value'], ascending=[True, False])
    cand_part['rank'] = cand_part.groupby('customer_id').cumcount() + 1
    cand_part = cand_part[cand_part['rank'] <= N_PER_USER][['customer_id', 'article_id', 'value']]

    out_parts.append(cand_part)
    del rb, cand_part
    gc.collect()

cand = cudf.concat(out_parts)
cand = cand.drop_duplicates(['customer_id', 'article_id'])
cand['window_type'] = 'itemcf'

cand.head()

,customer_id,article_id,value,window_type
4922602,-9223343869995384291,849886012,0.180579,itemcf
5139402,-9223343869995384291,868213012,0.090183,itemcf
2880039,-9223343869995384291,851370004,0.084177,itemcf
3481307,-9223343869995384291,851774008,0.077013,itemcf
4983855,-9223343869995384291,906305001,0.076734,itemcf


In [16]:
# Cell 8 – add customer_id_hex and save

# CHANGE: Move to Pandas (CPU) for the final merge to avoid GPU OOM
cand_pd = cand.to_pandas()
del cand
gc.collect()

# Load sample_submission in Pandas
customers = pd.read_csv(
    '../data/input_data/sample_submission.csv',
    usecols=['customer_id'],
    dtype={'customer_id': 'string'}
)
customers['customer_id_int'] = customers['customer_id'].str[-16:].apply(lambda h: int(h, 16)).astype('int64')

cand_pd = cand_pd.merge(
    customers[['customer_id', 'customer_id_int']],
    left_on='customer_id',
    right_on='customer_id_int',
    how='left'
)

cand_pd = cand_pd.rename(columns={'customer_id_x': 'customer_id', 'customer_id_y': 'customer_id_hex'})
cand_pd = cand_pd.drop('customer_id_int', axis=1)

# Types
cand_pd['article_id'] = cand_pd['article_id'].astype('int32')
cand_pd['value']      = cand_pd['value'].astype('float32')

# Note: drop_duplicates is technically redundant here because the generation logic 
# ensures uniqueness per user, but we keep it for safety (CPU can handle it).
cand_pd = cand_pd.drop_duplicates(['customer_id', 'article_id'])

# Save
cand_pd.to_parquet(f'../data/outputs/candidates/candidates_itemcf_{printdate}.parquet', index=False)

del tx, ui, pairs, cooc, item_freq, item_sim, recent, cand_pd, customers, tx_sorted
gc.collect()

22